# COP27
> Ce notebook sert de marche à suivre pour analyser les données d'une journée de COP

In [21]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import datetime

import sys
sys.path.append("../../")

%load_ext autoreload
%autoreload 2

from quotaclimat.utils.plotly_theme import *
from quotaclimat.data_analytics.exploration import filter_data_between_hours
from quotaclimat.utils.channels import TOP_25_CHANNELS,TOP_CHANNELS_TV,TOP_CHANNELS_TV_8
#from quotaclimat.data_analytics.exploration import show_mentions_by_channel
#from quotaclimat.data_analytics.exploration import show_mentions_by_time_of_the_day
#from quotaclimat.data_analytics.exploration import show_mentions_over_time
#from quotaclimat.data_analytics.exploration import show_mentions_treemap
#from quotaclimat.data_analytics.exploration import show_piechart_split_tv_radio

#from quotaclimat.data_processing.read_format_deduplicate import read_and_format_one
from quotaclimat.data_processing.read_format_deduplicate import read_and_format_all_data_dump
#from quotaclimat.data_processing.read_format_deduplicate import deduplicate_extracts
from quotaclimat.data_processing.keyword_processor import KeywordModel
from quotaclimat.data_processing.uncertainty_processing import compute_media_time, compute_chunk_score
from quotaclimat.data_processing.uncertainty_processing import get_coverage_by_chunks, get_coverage_by_window
from quotaclimat.data_processing.uncertainty_processing import get_mention_df_for_plot
from quotaclimat.data_processing.uncertainty_processing import get_mention_timestamp_by_channel , get_coverage_data_by_channel
from quotaclimat.data_processing.uncertainty_processing import plot_grouped_chunks_and_mentions

from tqdm.auto import tqdm

def make_ws_palette(n = 10):
    import seaborn as sns
    return [f"rgb({int(x*255)},{int(y*255)},{int(z*255)})" for x,y,z in list(sns.color_palette("RdBu_r",n_colors=n))]


from quotaclimat.utils.plotly_theme import WARMING_STRIPES_SEQUENCE

COLOR_RADIO = WARMING_STRIPES_SEQUENCE[0]
COLOR_TV= WARMING_STRIPES_SEQUENCE[1]
COLOR_ECO = WARMING_STRIPES_SEQUENCE[3]


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Récupération et préparation des données

## Récupérer et préparer les données de la journée
Choisissez bien le bon fichier

## Filtrer sur les 50 chaînes TV et Radio avec le plus d'audience

In [2]:
top_audiences = pd.read_excel("../../data/channels.xlsx",sheet_name = "top_audiences")
top_audiences.loc[top_audiences.channel_name == 'Sud Radio', 'channel_name'] = 'Sud Radio Paris'
top_audiences["channel_id"] = top_audiences["channel_name"] + "_" + top_audiences["media"]
top_channels_tv = top_audiences.query("media=='TV'")["channel_name"].tolist()
top_channels_radio = top_audiences.query("media=='Radio'")["channel_name"].tolist()

top_channels_tv_gen = top_audiences.query("type=='Généraliste'")["channel_name"].tolist()
top_channels_tv_info = top_audiences.query("type=='Information en continu'")["channel_name"].tolist()

In [3]:

data_all_days = []
for folder in tqdm(["05","06","07","08","09", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"]):
    
    day = f"{folder}11"

    data_total = read_and_format_all_data_dump(path_folder = f"../../data/COP27/{day}/",path_channel_metadata=None)

    data_total["channel_id"] = data_total["channel_name"] + "_" + data_total["media"]
    data_total = data_total.merge(top_audiences[["channel_id","type"]],on = ["channel_id"],how = "inner")

    data_total.loc[data_total["media"]=="TV","media2"] = "TV" + " - " + data_total["type"]
    data_total.loc[data_total["media"]=="Radio","media2"] = "Radio"

    data_total = filter_data_between_hours(data_total,"06:00","24:00").reset_index(drop = True)
    #data_total.loc[data_total["keywords"].map(lambda x : "cop27" in x),"keyword"] = "COP27"

    data_total = data_total[data_total["keyword"].isin(['COP27', 'ecologie'])]
    assert sorted(data_total["keyword"].unique()) == ["COP27","ecologie"]
    
    
    data_total["day_extract"] = day
    data_all_days.append(data_total)
    
data_all_days = pd.concat(data_all_days,axis = 0,ignore_index = True)

data_all_days["day_dt"] = data_all_days["date"].dt.date
data_all_days["day_str"] = data_all_days["day_dt"].map(str)

100%|██████████| 16/16 [00:37<00:00,  2.32s/it]


# Incertitudes

### Méthodologie d'incertitude

J'ai testé d'implémenter ça avec des valeurs plus solides que les 15 et 30s, en me basant l'échantillon de 138 extraits de 120s avec au moins une mention qui ont été cross-checké à la main. La part d'occupation du sujet a été très variable sur les 2min à chaque fois, et on a des quantiles q25 / q50 / q75 = 6 / 21 / 67s (à peine plus de la moitié des 2min pour 75% des extraits de l'échantillon donc). La moyenne est à 30,8s.

J'ai donc trois cas :
 - max : tous le chunck
 - nominal : la partie du chunck encadré par les mentions + 30.8s (la moyenne)
 - min : la partie du chunck encadré par les mentions + 6s (quantile à 25%)



Maximum :
* 1 chunk = 2min
* pas de marge méthode (On est déjà très généreux, pas sûr que ça soit utile)

Nominal :
* Si un seul highlight dans le chunk : moyenne = 30.8s (=0.257)
* Si plusieurs highlights : 15.4s (=0.128) ajouté à chaque extrémités des mentions, dans la limites de [0,1]

"Minimum" :
* Si un seul highlight dans le chunk : q25 = 6s (=0.025)
* Si plusieurs highlights : 3s (=0.0125) ajouté à chaque extrémités des mentions, dans la limites de [0,1]

C'est un peu violent de répéter à chaque fois le quantile qui est bas, mais je me dis que les tirages ne sont pas du tout indépendants à l'intérieur d'une chaine, donc on ne peut pas réduire non plus. On n'ajoute donc pas de marge méthode.

In [4]:
# Read keywords associated to the text extraction
keywords_dict = pd.read_excel("../../data/keywords.xlsx",sheet_name = None)
# format with list for extract_mentions()
keywords_dict = {k:list(v.squeeze("columns")) for k,v in keywords_dict.items() }

In [5]:
kw = KeywordModel(keywords_dict)

data = kw.extract_mentions(data_all_days)

In [6]:
continuity_window = datetime.timedelta(minutes=2)
mention_extension = datetime.timedelta(seconds=30.8)
channel_name = 'BFMTV'

In [7]:

cov_data_bfm = get_coverage_data_by_channel(data,channel_name)
print('Taille des données:',cov_data_bfm.shape)

mention_timestamp = get_mention_timestamp_by_channel(cov_data_bfm,channel_name)
df_event = get_mention_df_for_plot(mention_timestamp,channel_name)


Taille des données: (764, 8)


In [8]:
len(mention_timestamp)


1163

## quantification des temps

In [9]:
chunks_method_initiale = get_coverage_by_chunks(cov_data_bfm,channel_name)

In [22]:
chunks_method_nominal = get_coverage_by_window(cov_data_bfm,channel_name,
                    continuity_window_minutes= 2,
                    mention_extension_seconds= 30.8)

In [23]:
chunks_method_test = get_coverage_by_window(cov_data_bfm,channel_name,
                    continuity_window_minutes= 5,
                    mention_extension_seconds= 30.8)

In [24]:
results_media = dict()

chunk_name_list = ['Initial', 'Nominal','Extended_window']
chunk_list = [chunks_method_initiale,chunks_method_nominal,chunks_method_test]

for name,chunks in zip(chunk_name_list,chunk_list) :
    results_media[(name,'time')] = compute_media_time(chunks)
    results_media[(name,'score')] = compute_chunk_score(chunks)

In [25]:
results_media

{('Initial', 'time'): 0.06747685185185186,
 ('Initial', 'score'): 0.056399956096534934,
 ('Nominal', 'time'): 0.031572806914544754,
 ('Nominal', 'score'): 0.0340579408096274,
 ('Extended_window', 'time'): 0.0448229441097608,
 ('Extended_window', 'score'): 0.04801165605245081}

In [26]:
results_media_df = pd.DataFrame(results_media,index=[0]).T
#results_media_df.pivot_table(columns=chunk_name_list)
results_media_df = results_media_df.reset_index().pivot(index='level_0',columns='level_1')
results_media_df.columns = ['score','time']
results_media_df

,score,time
level_0,,
Extended_window,0.048012,0.044823
Initial,0.056400,0.067477
Nominal,0.034058,0.031573


## Plot

## Regroupement des mentions

In [27]:
fig_intiale = plot_grouped_chunks_and_mentions(chunks_method_initiale,df_event)
fig_intiale.show()


In [28]:
fig_intiale.write_html("../../../mentions_BFM_COP27_regrouped.html")

## Regroupement en continu

In [29]:
fig_continuous = plot_grouped_chunks_and_mentions(chunks_method_nominal,df_event)
fig_continuous.show()


In [30]:
fig_continuous.write_html("../../../mentions_BFM_COP27_continuous_chunks_nominal.html")

In [31]:
fig_continuous_extended = plot_grouped_chunks_and_mentions(chunks_method_test,df_event)
fig_continuous_extended.show()

In [32]:
fig_continuous_extended.write_html("../../../mentions_BFM_COP27_continuous_chunks_extended.html")